In [1]:
import sys
sys.path.append('G:\我的雲端硬碟\python')
import numpy as np
from polib.CsvEngn import *
from MLB_functions import *
from tqdm import tqdm

import statsmodels.api as sm #OLS
# https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html

from linearmodels.panel import PanelOLS # FE
# https://bashtage.github.io/linearmodels/doc/panel/models.html

# 讀檔

In [13]:
#野手
df_bat = pd.read_excel(r"G:\我的雲端硬碟\python\output\0904\Spotrac_bat_data_20210904_21.22.13.xlsx",
                        engine='openpyxl')
# 投手
df_pitch= pd.read_excel(r"G:\我的雲端硬碟\python\output\0904\Spotrac_pitch_data_20210904_21.22.17.xlsx",
                      engine='openpyxl')

# 特徵轉換

In [14]:
# # is_2020+2021
df_bat.loc[:,"is_gt_2020"] = (df_bat.loc[:,"Season"]>=2020)
df_pitch.loc[:,"is_gt_2020"] = (df_pitch.loc[:,"Season"]>=2020)

In [15]:
# for df in [df_pitch, df_bat]:
for df in [df_pitch,df_bat]:
    
    #T,F transform
    for col in [col for col in df_bat.columns if "is_" in col]:
        df.loc[:,col] = df.loc[:,col].apply(lambda v: 1 if v else 0)
        
    # AAV_dollar 取 自然對數
    df.loc[:,"AAV_dollar"] = np.log(df.loc[:,"AAV_dollar"])
    df.loc[np.isinf(df_bat.loc[:,"AAV_dollar"]),"AAV_dollar"] = np.nan
    
#     NewCBA 2017~2021 dummy
    df.loc[:,"NewCBA"] =df.loc[:,"Season"].apply(lambda s: 1 if (int(s)>=2017) & (int(s)<=2021) else 0)
    

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# 年度篩選

In [83]:
# 篩選
df_pitch = df_pitch[(df_pitch.loc[:,"Season"]<=2021) & (df_pitch.loc[:,"Season"]>=2012)]
print(len(df_pitch))
# df_pitch = df_pitch[(df_pitch.loc[:,"Season"]<=2016) & (df_pitch.loc[:,"Season"]>=2012)]
# df_pitch = df_pitch[(df_pitch.loc[:,"Season"]>=2017) & (df_pitch.loc[:,"Season"]<=2021)]
print(len(df_pitch))

2550
2550


# Model1

In [ ]:
#     首先以合約剩餘年數作為主要的解釋變數，提出的普通最小平方法模型 (Ordinary Least Squares Method) 如下：

# WAR = β1 Contract Years Remaining + One Year Contract + Age +  Age2 + Position + ln (Average Salary) + Team + ChangedTeam + Last Year + 2020season + 2021season + ε 
# ……… (1)


##  投手

In [16]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2550
940


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     8.467
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           9.94e-39
Time:                        16:41:09   Log-Likelihood:                -1564.7
No. Observations:                 940   AIC:                             3207.
Df Residuals:                     901   BIC:                             3396.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.9892      1.190     -0.831      0.406      -3.325       1.346
is_changeTeam        -0.4471      0.109     -4.109      0.000      -0.661      -0.234
Age                   0.0020      0.016      0.120      0.904      -0.030       0.034
Age^2             -1.014e-06   8.02e-06     -0.127      0.899   -1.67e-05    1.47e-05
YRS_left              0.1974      0.048      4.110      0.000       0.103       0.292
AAV_dollar            0.0919      0.071      1.287      0.199      -0.048       0.232
is_1year_contract    -0.0160      0.128     -0.125      0.901      -0.267       0.235
is_END_year          -0.9493      0.141     -6.750      0.000      -1.225      -0.673
Team_ATL              0.4412      0.354      1.246      0.213      -0.254       1.136
Team_BAL             -0.4193      0.475     -0.883      0.377      -1.351       0.512
Team_BOS              0.5997      0.338      1.774      0.076      -0.064       1.263
Team_CHC              0.0865      0.325      0.266      0.790      -0.552       0.725
Team_CHW              0.6073      0.349      1.738      0.083      -0.078       1.293
Team_CIN             -0.1051      0.349     -0.301      0.764      -0.791       0.581
Team_CLE              0.5362      0.375      1.432      0.153      -0.199       1.271
Team_COL             -0.3242      0.371     -0.873      0.383      -1.053       0.404
Team_DET             -0.1823      0.359     -0.508      0.611      -0.886       0.522
Team_HOU              0.4129      0.339      1.217      0.224      -0.253       1.079
Team_KC              -0.0416      0.342     -0.122      0.903      -0.712       0.629
Team_LAA             -0.3169      0.362     -0.876      0.381      -1.027       0.393
Team_LAD             -0.0062      0.330     -0.019      0.985      -0.654       0.642
Team_MIA             -0.2192      0.376     -0.583      0.560      -0.956       0.518
Team_MIL             -0.0397      0.342     -0.116      0.908      -0.711       0.631
Team_MIN             -0.1200      0.341     -0.351      0.725      -0.790       0.550
Team_NYM             -0.1884      0.344     -0.547      0.585      -0.864       0.488
Team_NYY              0.2818      0.356      0.791      0.429      -0.418       0.981
Team_OAK              0.6236      0.346      1.803      0.072      -0.055       1.302
Team_PHI              0.4941      0.347      1.424      0.155      -0.187       1.175
Team_PIT              0.4627      0.438      1.056      0.291      -0.397       1.323
Team_SD               0.0428      0.356      0.120      0.904      -0.656       0.741
Team_SEA              0.3267      0.361      0.906      0.365      -0.381       1.034
Team_SF              -0.1068      0.333     -0.321      0.748      -0.760       0.547
Team_STL             -0.1344      0.364     -0.369      0.712      -0.849

## 野手

In [17]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2567
970


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.315
Method:                 Least Squares   F-statistic:                     10.69
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.38e-58
Time:                        16:41:09   Log-Likelihood:                -1772.2
No. Observations:                 970   AIC:                             3638.
Df Residuals:                     923   BIC:                             3868.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.5536      1.326     -0.417      0.677      -3.157       2.050
is_changeTeam        -0.6808      0.131     -5.201      0.000      -0.938      -0.424
Age                  -0.1120      0.020     -5.718      0.000      -0.150      -0.074
Age^2              5.451e-05   9.56e-06      5.704      0.000    3.58e-05    7.33e-05
YRS_left              0.1110      0.035      3.184      0.002       0.043       0.179
AAV_dollar            0.3909      0.076      5.155      0.000       0.242       0.540
is_1year_contract     0.2185      0.154      1.419      0.156      -0.084       0.521
is_END_year          -1.2629      0.161     -7.838      0.000      -1.579      -0.947
Team_ATL             -0.7693      0.350     -2.201      0.028      -1.455      -0.083
Team_BAL             -1.0052      0.423     -2.378      0.018      -1.835      -0.176
Team_BOS             -0.4855      0.343     -1.415      0.157      -1.159       0.188
Team_CHC             -0.6513      0.374     -1.740      0.082      -1.386       0.083
Team_CHW             -0.5807      0.392     -1.481      0.139      -1.350       0.189
Team_CIN             -0.4458      0.378     -1.180      0.238      -1.187       0.296
Team_CLE             -0.7296      0.349     -2.092      0.037      -1.414      -0.045
Team_COL             -1.4882      0.399     -3.726      0.000      -2.272      -0.704
Team_DET             -0.8209      0.396     -2.073      0.038      -1.598      -0.044
Team_HOU              0.4059      0.382      1.063      0.288      -0.344       1.155
Team_KC              -0.6409      0.401     -1.598      0.110      -1.428       0.146
Team_LAA             -0.4290      0.351     -1.224      0.221      -1.117       0.259
Team_LAD              0.4369      0.394      1.109      0.268      -0.336       1.210
Team_MIA             -0.6829      0.384     -1.778      0.076      -1.437       0.071
Team_MIL              0.1431      0.367      0.390      0.697      -0.578       0.864
Team_MIN             -0.6055      0.421     -1.437      0.151      -1.432       0.221
Team_NYM             -0.4792      0.365     -1.312      0.190      -1.196       0.238
Team_NYY             -0.2110      0.355     -0.594      0.553      -0.908       0.486
Team_OAK             -0.2196      0.411     -0.534      0.593      -1.026       0.587
Team_PHI             -0.5170      0.402     -1.287      0.198      -1.306       0.271
Team_PIT             -0.4510      0.429     -1.051      0.294      -1.293       0.391
Team_SD              -0.8867      0.403     -2.201      0.028      -1.677      -0.096
Team_SEA              0.2619      0.403      0.650      0.516      -0.529       1.053
Team_SF              -0.9007      0.383     -2.351      0.019      -1.653      -0.149
Team_STL             -0.1090      0.365     -0.298      0.766      -0.826

### 前5年

In [32]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2012 2013 2014 2015 2016]
1633
458


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.315
Method:                 Least Squares   F-statistic:                     5.661
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           7.78e-23
Time:                        16:50:37   Log-Likelihood:                -852.65
No. Observations:                 458   AIC:                             1797.
Df Residuals:                     412   BIC:                             1987.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -1.4233      2.089     -0.681      0.496      -5.530       2.683
is_changeTeam        -0.7868      0.206     -3.822      0.000      -1.191      -0.382
Age                  -0.1139      0.032     -3.541      0.000      -0.177      -0.051
Age^2              5.547e-05   1.57e-05      3.532      0.000    2.46e-05    8.63e-05
YRS_left              0.1316      0.063      2.081      0.038       0.007       0.256
AAV_dollar            0.4700      0.122      3.838      0.000       0.229       0.711
is_1year_contract     0.2040      0.249      0.819      0.413      -0.286       0.694
is_END_year          -1.1460      0.260     -4.408      0.000      -1.657      -0.635
Team_ATL             -1.4722      0.588     -2.503      0.013      -2.629      -0.316
Team_BAL             -0.2952      0.725     -0.407      0.684      -1.720       1.130
Team_BOS             -0.5068      0.565     -0.896      0.371      -1.618       0.605
Team_CHC             -0.9635      0.604     -1.595      0.111      -2.151       0.224
Team_CHW             -1.0061      0.620     -1.623      0.105      -2.225       0.212
Team_CIN             -0.9234      0.582     -1.587      0.113      -2.067       0.221
Team_CLE             -1.3709      0.584     -2.346      0.019      -2.520      -0.222
Team_COL             -1.2835      0.671     -1.913      0.056      -2.602       0.035
Team_DET             -0.8262      0.634     -1.304      0.193      -2.072       0.419
Team_HOU             -0.0616      0.714     -0.086      0.931      -1.465       1.341
Team_KC              -1.4309      0.719     -1.989      0.047      -2.845      -0.017
Team_LAA             -0.3975      0.618     -0.643      0.521      -1.613       0.818
Team_LAD              0.4154      0.701      0.592      0.554      -0.963       1.794
Team_MIA             -1.1146      0.587     -1.898      0.058      -2.269       0.040
Team_MIL             -0.3445      0.601     -0.573      0.567      -1.527       0.838
Team_MIN             -1.1839      0.680     -1.742      0.082      -2.520       0.152
Team_NYM             -0.1766      0.644     -0.274      0.784      -1.442       1.089
Team_NYY             -1.0654      0.560     -1.901      0.058      -2.167       0.036
Team_OAK             -1.1634      0.678     -1.715      0.087      -2.497       0.170
Team_PHI             -0.4902      0.752     -0.652      0.515      -1.968       0.987
Team_PIT             -0.5336      0.637     -0.838      0.402      -1.785       0.718
Team_SD              -1.2536      0.655     -1.915      0.056      -2.540       0.033
Team_SEA             -0.3194      0.613     -0.521      0.602      -1.524       0.885
Team_SF              -1.1885      0.670     -1.775      0.077      -2.505       0.128
Team_STL             -0.0573      0.600     -0.096      0.924      -1.236

### 後5年

In [33]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2017 2018 2019 2020 2021]
934
512


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                     6.500
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           2.72e-28
Time:                        16:51:12   Log-Likelihood:                -883.80
No. Observations:                 512   AIC:                             1862.
Df Residuals:                     465   BIC:                             2061.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.2930      1.842      0.702      0.483      -2.326       4.912
is_changeTeam        -0.5579      0.172     -3.243      0.001      -0.896      -0.220
Age                  -0.1685      0.030     -5.709      0.000      -0.227      -0.111
Age^2              8.198e-05   1.44e-05      5.698      0.000    5.37e-05       0.000
YRS_left              0.0554      0.045      1.226      0.221      -0.033       0.144
AAV_dollar            0.3828      0.107      3.579      0.000       0.173       0.593
is_1year_contract     0.2092      0.199      1.054      0.293      -0.181       0.599
is_END_year          -1.1317      0.210     -5.383      0.000      -1.545      -0.719
Team_ATL             -0.1959      0.428     -0.458      0.647      -1.037       0.645
Team_BAL             -1.3781      0.513     -2.687      0.007      -2.386      -0.370
Team_BOS             -0.6323      0.430     -1.472      0.142      -1.477       0.212
Team_CHC             -0.3898      0.484     -0.805      0.421      -1.341       0.561
Team_CHW             -0.2140      0.525     -0.407      0.684      -1.246       0.818
Team_CIN              0.0135      0.539      0.025      0.980      -1.045       1.072
Team_CLE             -0.2521      0.438     -0.576      0.565      -1.113       0.609
Team_COL             -1.5511      0.489     -3.170      0.002      -2.513      -0.590
Team_DET             -1.0200      0.526     -1.940      0.053      -2.053       0.013
Team_HOU              0.8792      0.445      1.978      0.049       0.006       1.753
Team_KC              -0.2287      0.471     -0.485      0.628      -1.155       0.697
Team_LAA             -0.2760      0.423     -0.653      0.514      -1.107       0.555
Team_LAD              0.7265      0.469      1.549      0.122      -0.195       1.648
Team_MIA             -0.3870      0.537     -0.721      0.472      -1.443       0.669
Team_MIL              0.5656      0.461      1.228      0.220      -0.340       1.471
Team_MIN             -0.1013      0.545     -0.186      0.853      -1.172       0.969
Team_NYM             -0.5281      0.435     -1.215      0.225      -1.382       0.326
Team_NYY              0.6124      0.473      1.295      0.196      -0.317       1.542
Team_OAK              0.3923      0.522      0.752      0.452      -0.633       1.417
Team_PHI             -0.3216      0.470     -0.685      0.494      -1.245       0.601
Team_PIT             -0.7219      0.631     -1.144      0.253      -1.962       0.518
Team_SD              -0.6208      0.524     -1.184      0.237      -1.651       0.410
Team_SEA              0.8988      0.577      1.556      0.120      -0.236       2.034
Team_SF              -0.6374      0.462     -1.380      0.168      -1.545       0.270
Team_STL             -0.0679      0.461     -0.147      0.883      -0.974

# Model 2

In [18]:
#     為了控制上述的問題，在模型 (2) 改以固定效果模型 (Fixed Effect Model)取代原先的 OLS 模型：


# WARit = β1 Contract Years Remainingit +β2 One Year Contractit + Ageit + Ageit2 + Positionit + ln (Average Salary)it + Teamit + ChangedTeamit + Last Yearit +  2020seasonit +  2021seasonit + αi + λt + εit 
# ……… (2)


## 投手

In [19]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2550
940


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_SP

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.1798
Estimator:                   PanelOLS   R-squared (Between):             -8991.6
No. Observations:                 940   R-squared (Within):               0.1798
Date:                Sun, Sep 05 2021   R-squared (Overall):             -4305.8
Time:                        16:41:09   Log-likelihood                   -1310.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.2652
Entities:                         352   P-value                           0.0000
Avg Obs:                       2.6705   Distribution:                  F(37,551)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             5.0817
                            

## 野手

In [20]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2567
970
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3231
Estimator:                   PanelOLS   R-squared (Between):          -1.082e+05
No. Observations:                 970   R-squared (Within):               0.3231
Date:                Sun, Sep 05 2021   R-squared (Overall):          -4.265e+04
Time:                        16:41:09   Log-likelihood                   -1453.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      7.6102
Entities:                         343   P-value                           0.0000
Avg Obs:                       2.8280   Distribution:                  F(37,590)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statis

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS

  warnings.warn(


### 前5年

In [34]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016]
1633
458
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.2474
Estimator:                   PanelOLS   R-squared (Between):          -1.582e+05
No. Observations:                 458   R-squared (Within):               0.2474
Date:                Sun, Sep 05 2021   R-squared (Overall):          -1.126e+05
Time:                        16:53:07   Log-likelihood                   -650.06
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.9627
Entities:                         207   P-value                           0.0018
Avg Obs:                       2.2126   Distribution:                  F(36,215)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):          5.

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS, is_gt_2020

  warnings.warn(


### 後5年

In [35]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2017 2018 2019 2020 2021]
934
512
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3769
Estimator:                   PanelOLS   R-squared (Between):          -5.253e+04
No. Observations:                 512   R-squared (Within):               0.3769
Date:                Sun, Sep 05 2021   R-squared (Overall):          -2.435e+04
Time:                        16:53:18   Log-likelihood                   -678.86
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      4.1196
Entities:                         223   P-value                           0.0000
Avg Obs:                       2.2960   Distribution:                  F(37,252)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS

  warnings.warn(


# Model 3

In [21]:
# 模型 (3) 將主要的解釋變數由線性的合約剩餘年數替換為合約剩餘兩年、合約剩餘三年、合約剩餘四年以上的虛擬變數：

# WARit = Two Years Remainingit + Three Years Remainingit + FourPlus Years Remainingit + One Year Contractit + Ageit + Ageit2 + Positionit + ln (Average Salary)it + Teamit + ChangedTeamit + Last Yearit +  2020seasonit +  2021seasonit + αi + λt + εit 
# ……… (3)


## 投手

In [22]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2550
940
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.1826
Estimator:                   PanelOLS   R-squared (Between):             -8297.0
No. Observations:                 940   R-squared (Within):               0.1826
Date:                Sun, Sep 05 2021   R-squared (Overall):             -3973.3
Time:                        16:41:10   Log-likelihood                   -1309.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.1439
Entities:                         352   P-value                           0.0000
Avg Obs:                       2.6705   Distribution:                  F(39,549)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statis

## 野手

In [23]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2567
970
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3306
Estimator:                   PanelOLS   R-squared (Between):          -1.133e+05
No. Observations:                 970   R-squared (Within):               0.3306
Date:                Sun, Sep 05 2021   R-squared (Overall):          -4.465e+04
Time:                        16:41:43   Log-likelihood                   -1448.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      7.4449
Entities:                         343   P-value                           0.0000
Avg Obs:                       2.8280   Distribution:                  F(39,588)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statis

### 前5年

In [36]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016]
1633
458
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.2419
Estimator:                   PanelOLS   R-squared (Between):          -1.072e+05
No. Observations:                 458   R-squared (Within):               0.2419
Date:                Sun, Sep 05 2021   R-squared (Overall):          -7.637e+04
Time:                        16:53:39   Log-likelihood                   -651.71
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.7889
Entities:                         207   P-value                           0.0055
Avg Obs:                       2.2126   Distribution:                  F(38,213)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):          1.

### 後5年

In [37]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2017 2018 2019 2020 2021]
934
512
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3812
Estimator:                   PanelOLS   R-squared (Between):          -6.198e+04
No. Observations:                 512   R-squared (Within):               0.3812
Date:                Sun, Sep 05 2021   R-squared (Overall):          -2.873e+04
Time:                        16:53:49   Log-likelihood                   -677.10
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.9485
Entities:                         223   P-value                           0.0000
Avg Obs:                       2.2960   Distribution:                  F(39,250)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             

# Model 4

In [6]:
# SHIRK = β1 First Year + One Year Contract + Age + Age2 + Position + ln (Average Salary) + Team + ChangedTeam + Last Year + 2020season + 2021season + ε……… (4)

## 投手

In [28]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2550
940


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.250
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     7.901
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.17e-35
Time:                        16:46:32   Log-Likelihood:                -1573.1
No. Observations:                 940   AIC:                             3224.
Df Residuals:                     901   BIC:                             3413.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -1.0395      1.202     -0.865      0.387      -3.398       1.319
is_changeTeam                -0.5884      0.104     -5.642      0.000      -0.793      -0.384
Age                          -0.0142      0.016     -0.882      0.378      -0.046       0.017
Age^2                      6.864e-06   7.84e-06      0.875      0.382   -8.53e-06    2.23e-05
AAV_dollar                    0.1592      0.070      2.272      0.023       0.022       0.297
is_1year_contract            -0.1336      0.137     -0.976      0.329      -0.402       0.135
is_FIRST_year_BigContract    -0.1035      0.118     -0.875      0.382      -0.336       0.129
is_END_year                  -1.1106      0.136     -8.153      0.000      -1.378      -0.843
Team_ATL                      0.4147      0.357      1.161      0.246      -0.286       1.116
Team_BAL                     -0.3526      0.479     -0.737      0.462      -1.292       0.587
Team_BOS                      0.5544      0.341      1.626      0.104      -0.115       1.223
Team_CHC                      0.1356      0.328      0.413      0.679      -0.508       0.779
Team_CHW                      0.5854      0.352      1.661      0.097      -0.106       1.277
Team_CIN                     -0.0535      0.352     -0.152      0.879      -0.745       0.638
Team_CLE                      0.5207      0.378      1.378      0.169      -0.221       1.262
Team_COL                     -0.3334      0.375     -0.890      0.374      -1.069       0.402
Team_DET                     -0.1418      0.362     -0.392      0.695      -0.852       0.568
Team_HOU                      0.3862      0.342      1.129      0.259      -0.285       1.058
Team_KC                      -0.0555      0.345     -0.161      0.872      -0.732       0.621
Team_LAA                     -0.3275      0.365     -0.897      0.370      -1.044       0.389
Team_LAD                     -0.0223      0.333     -0.067      0.947      -0.676       0.631
Team_MIA                     -0.1063      0.378     -0.281      0.779      -0.848       0.635
Team_MIL                     -0.0552      0.345     -0.160      0.873      -0.732       0.622
Team_MIN                     -0.1271      0.344     -0.369      0.712      -0.803       0.549
Team_NYM                     -0.1944      0.347     -0.560      0.576      -0.876       0.488
Team_NYY                      0.3546      0.359      0.987      0.324      -0.351       1.060
Team_OAK                      0.5685      0.349      1.631      0.103      -0.116       1.253
Team_PHI                      0.4852      0.350      1.385      0.166      -0.202       1.173
Team_PIT                      0.4872      0.442      1.102      0.271      -0.380       1.355
Team_SD                       0.0670      0.359      0.186      0.852      

## 野手

In [29]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2567
970


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     10.37
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.14e-56
Time:                        16:47:03   Log-Likelihood:                -1777.3
No. Observations:                 970   AIC:                             3649.
Df Residuals:                     923   BIC:                             3878.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.9479      1.332     -0.712      0.477      -3.562       1.667
is_changeTeam                -0.8122      0.127     -6.395      0.000      -1.061      -0.563
Age                          -0.1321      0.019     -7.091      0.000      -0.169      -0.096
Age^2                      6.435e-05   9.09e-06      7.077      0.000    4.65e-05    8.22e-05
AAV_dollar                    0.4750      0.072      6.621      0.000       0.334       0.616
is_1year_contract             0.2204      0.162      1.357      0.175      -0.098       0.539
is_FIRST_year_BigContract     0.0813      0.135      0.601      0.548      -0.184       0.347
is_END_year                  -1.3741      0.159     -8.652      0.000      -1.686      -1.062
Team_ATL                     -0.7850      0.351     -2.234      0.026      -1.475      -0.095
Team_BAL                     -1.0343      0.425     -2.435      0.015      -1.868      -0.201
Team_BOS                     -0.5224      0.345     -1.515      0.130      -1.199       0.154
Team_CHC                     -0.5744      0.376     -1.528      0.127      -1.312       0.163
Team_CHW                     -0.6084      0.394     -1.543      0.123      -1.382       0.165
Team_CIN                     -0.3373      0.378     -0.892      0.373      -1.080       0.405
Team_CLE                     -0.7413      0.351     -2.114      0.035      -1.429      -0.053
Team_COL                     -1.5424      0.401     -3.843      0.000      -2.330      -0.755
Team_DET                     -0.7444      0.397     -1.874      0.061      -1.524       0.035
Team_HOU                      0.4022      0.384      1.048      0.295      -0.351       1.156
Team_KC                      -0.6245      0.403     -1.548      0.122      -1.416       0.167
Team_LAA                     -0.3270      0.351     -0.932      0.352      -1.016       0.362
Team_LAD                      0.5261      0.395      1.331      0.184      -0.250       1.302
Team_MIA                     -0.6453      0.386     -1.671      0.095      -1.403       0.113
Team_MIL                      0.1005      0.369      0.272      0.785      -0.624       0.825
Team_MIN                     -0.6538      0.423     -1.544      0.123      -1.485       0.177
Team_NYM                     -0.4524      0.367     -1.232      0.218      -1.173       0.268
Team_NYY                     -0.2141      0.357     -0.599      0.549      -0.915       0.487
Team_OAK                     -0.3002      0.413     -0.728      0.467      -1.110       0.509
Team_PHI                     -0.4534      0.403     -1.124      0.261      -1.245       0.338
Team_PIT                     -0.5309      0.431     -1.232      0.218      -1.376       0.315
Team_SD                      -0.7333      0.402     -1.822      0.069      

### 前5年

In [38]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016]
1633
458


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     5.593
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.72e-22
Time:                        16:54:11   Log-Likelihood:                -853.71
No. Observations:                 458   AIC:                             1799.
Df Residuals:                     412   BIC:                             1989.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.0494      2.087     -0.982      0.327      -6.151       2.052
is_changeTeam                -1.0026      0.203     -4.948      0.000      -1.401      -0.604
Age                          -0.1377      0.030     -4.586      0.000      -0.197      -0.079
Age^2                      6.715e-05   1.47e-05      4.578      0.000    3.83e-05     9.6e-05
AAV_dollar                    0.5752      0.113      5.103      0.000       0.354       0.797
is_1year_contract             0.3118      0.267      1.168      0.243      -0.213       0.836
is_FIRST_year_BigContract     0.3234      0.208      1.554      0.121      -0.086       0.733
is_END_year                  -1.3276      0.259     -5.133      0.000      -1.836      -0.819
Team_ATL                     -1.3745      0.589     -2.335      0.020      -2.532      -0.217
Team_BAL                     -0.2980      0.727     -0.410      0.682      -1.726       1.130
Team_BOS                     -0.5090      0.567     -0.898      0.370      -1.623       0.605
Team_CHC                     -0.7881      0.603     -1.308      0.192      -1.973       0.397
Team_CHW                     -0.9641      0.622     -1.550      0.122      -2.186       0.258
Team_CIN                     -0.7749      0.579     -1.338      0.182      -1.913       0.363
Team_CLE                     -1.2856      0.584     -2.200      0.028      -2.434      -0.137
Team_COL                     -1.2940      0.672     -1.924      0.055      -2.616       0.028
Team_DET                     -0.6300      0.628     -1.003      0.317      -1.865       0.605
Team_HOU                      0.0125      0.716      0.017      0.986      -1.394       1.419
Team_KC                      -1.3982      0.721     -1.940      0.053      -2.815       0.019
Team_LAA                     -0.1601      0.609     -0.263      0.793      -1.358       1.038
Team_LAD                      0.4034      0.703      0.574      0.566      -0.978       1.785
Team_MIA                     -1.0161      0.587     -1.731      0.084      -2.170       0.138
Team_MIL                     -0.4170      0.601     -0.694      0.488      -1.599       0.765
Team_MIN                     -1.1767      0.681     -1.727      0.085      -2.516       0.162
Team_NYM                     -0.0634      0.645     -0.098      0.922      -1.331       1.204
Team_NYY                     -1.0099      0.561     -1.800      0.073      -2.113       0.093
Team_OAK                     -1.1829      0.680     -1.740      0.083      -2.519       0.153
Team_PHI                     -0.4227      0.753     -0.561      0.575      -1.904       1.058
Team_PIT                     -0.5983      0.637     -0.939      0.348      -1.851       0.654
Team_SD                      -1.1427      0.656     -1.742      0.082      

### 後5年

In [39]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2017 2018 2019 2020 2021]
934
512


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                     6.491
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           3.05e-28
Time:                        16:54:20   Log-Likelihood:                -883.94
No. Observations:                 512   AIC:                             1862.
Df Residuals:                     465   BIC:                             2061.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         1.1963      1.836      0.651      0.515      -2.412       4.805
is_changeTeam                -0.6069      0.166     -3.648      0.000      -0.934      -0.280
Age                          -0.1822      0.027     -6.635      0.000      -0.236      -0.128
Age^2                      8.867e-05   1.34e-05      6.624      0.000    6.24e-05       0.000
AAV_dollar                    0.4297      0.099      4.344      0.000       0.235       0.624
is_1year_contract             0.1478      0.205      0.721      0.471      -0.255       0.551
is_FIRST_year_BigContract    -0.2004      0.180     -1.113      0.266      -0.554       0.154
is_END_year                  -1.1753      0.208     -5.663      0.000      -1.583      -0.767
Team_ATL                     -0.2448      0.428     -0.572      0.567      -1.085       0.596
Team_BAL                     -1.4117      0.513     -2.751      0.006      -2.420      -0.403
Team_BOS                     -0.6731      0.429     -1.568      0.118      -1.517       0.170
Team_CHC                     -0.3896      0.484     -0.805      0.421      -1.341       0.562
Team_CHW                     -0.2256      0.525     -0.430      0.668      -1.258       0.807
Team_CIN                      0.0308      0.538      0.057      0.954      -1.027       1.089
Team_CLE                     -0.3107      0.438     -0.710      0.478      -1.171       0.550
Team_COL                     -1.5582      0.489     -3.185      0.002      -2.520      -0.597
Team_DET                     -1.0253      0.526     -1.949      0.052      -2.059       0.009
Team_HOU                      0.8610      0.445      1.935      0.054      -0.013       1.735
Team_KC                      -0.2448      0.472     -0.519      0.604      -1.172       0.682
Team_LAA                     -0.2495      0.422     -0.591      0.555      -1.079       0.580
Team_LAD                      0.7735      0.466      1.660      0.098      -0.142       1.689
Team_MIA                     -0.4166      0.538     -0.775      0.439      -1.474       0.640
Team_MIL                      0.5292      0.461      1.148      0.251      -0.376       1.435
Team_MIN                     -0.1211      0.545     -0.222      0.824      -1.191       0.949
Team_NYM                     -0.5354      0.435     -1.231      0.219      -1.390       0.319
Team_NYY                      0.5624      0.474      1.187      0.236      -0.369       1.494
Team_OAK                      0.3776      0.521      0.724      0.469      -0.647       1.402
Team_PHI                     -0.2980      0.469     -0.635      0.526      -1.220       0.624
Team_PIT                     -0.7362      0.631     -1.167      0.244      -1.976       0.503
Team_SD                      -0.5290      0.516     -1.025      0.306      

# Model 5 

In [7]:
# SHIRK = β1 NewCBA + One Year Contract + Age + Age2 + Position + ln (Average Salary) + Team + ChangedTeam + Last Year + 2020season + 2021season + ε……… (5)

## 投手

In [30]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2550
940


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     7.877
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.59e-35
Time:                        16:48:30   Log-Likelihood:                -1573.4
No. Observations:                 940   AIC:                             3225.
Df Residuals:                     901   BIC:                             3414.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -1.1159      1.204     -0.927      0.354      -3.478       1.247
is_changeTeam        -0.6034      0.103     -5.866      0.000      -0.805      -0.402
Age                  -0.0153      0.016     -0.942      0.346      -0.047       0.017
Age^2              7.408e-06   7.92e-06      0.936      0.350   -8.13e-06    2.29e-05
AAV_dollar            0.1654      0.071      2.324      0.020       0.026       0.305
is_1year_contract    -0.0895      0.128     -0.701      0.484      -0.340       0.161
is_END_year          -1.1202      0.137     -8.190      0.000      -1.389      -0.852
Team_ATL              0.4142      0.357      1.159      0.247      -0.287       1.115
Team_BAL             -0.3398      0.479     -0.710      0.478      -1.280       0.600
Team_BOS              0.5490      0.341      1.610      0.108      -0.120       1.218
Team_CHC              0.1356      0.328      0.413      0.680      -0.509       0.780
Team_CHW              0.5818      0.353      1.650      0.099      -0.110       1.274
Team_CIN             -0.0607      0.352     -0.172      0.863      -0.752       0.631
Team_CLE              0.5249      0.378      1.387      0.166      -0.218       1.268
Team_COL             -0.3357      0.375     -0.895      0.371      -1.071       0.400
Team_DET             -0.1407      0.362     -0.389      0.698      -0.851       0.570
Team_HOU              0.3835      0.342      1.120      0.263      -0.288       1.055
Team_KC              -0.0605      0.345     -0.175      0.861      -0.738       0.617
Team_LAA             -0.3350      0.365     -0.917      0.359      -1.052       0.382
Team_LAD             -0.0200      0.333     -0.060      0.952      -0.674       0.634
Team_MIA             -0.0945      0.378     -0.250      0.803      -0.836       0.647
Team_MIL             -0.0512      0.345     -0.148      0.882      -0.729       0.626
Team_MIN             -0.1312      0.344     -0.381      0.703      -0.807       0.545
Team_NYM             -0.1941      0.348     -0.558      0.577      -0.876       0.488
Team_NYY              0.3477      0.359      0.967      0.334      -0.358       1.053
Team_OAK              0.5698      0.349      1.630      0.103      -0.116       1.256
Team_PHI              0.4728      0.350      1.350      0.177      -0.214       1.160
Team_PIT              0.4807      0.442      1.087      0.277      -0.387       1.349
Team_SD               0.0585      0.359      0.163      0.871      -0.646       0.763
Team_SEA              0.2562      0.364      0.704      0.481      -0.458       0.970
Team_SF              -0.0858      0.336     -0.255      0.798      -0.745       0.574
Team_STL             -0.1099      0.367     -0.299      0.765      -0.831       0.611
Team_TB               0.0317      0.410      0.077      0.938      -0.773

## 野手

In [31]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2567
970


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     10.63
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           3.12e-58
Time:                        16:49:00   Log-Likelihood:                -1773.1
No. Observations:                 970   AIC:                             3640.
Df Residuals:                     923   BIC:                             3869.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -1.2253      1.328     -0.922      0.357      -3.832       1.382
is_changeTeam        -0.7879      0.125     -6.280      0.000      -1.034      -0.542
Age                  -0.1378      0.019     -7.387      0.000      -0.174      -0.101
Age^2              6.712e-05    9.1e-06      7.372      0.000    4.93e-05     8.5e-05
AAV_dollar            0.5178      0.073      7.090      0.000       0.374       0.661
is_1year_contract     0.2241      0.154      1.452      0.147      -0.079       0.527
is_END_year          -1.3197      0.159     -8.314      0.000      -1.631      -1.008
Team_ATL             -0.8191      0.350     -2.341      0.019      -1.506      -0.132
Team_BAL             -1.0461      0.423     -2.473      0.014      -1.876      -0.216
Team_BOS             -0.5752      0.344     -1.673      0.095      -1.250       0.100
Team_CHC             -0.6484      0.375     -1.730      0.084      -1.384       0.087
Team_CHW             -0.7197      0.394     -1.825      0.068      -1.494       0.054
Team_CIN             -0.4369      0.378     -1.155      0.248      -1.179       0.305
Team_CLE             -0.7968      0.349     -2.280      0.023      -1.483      -0.111
Team_COL             -1.5333      0.400     -3.838      0.000      -2.317      -0.749
Team_DET             -0.8239      0.397     -2.078      0.038      -1.602      -0.046
Team_HOU              0.3815      0.382      0.998      0.319      -0.369       1.132
Team_KC              -0.6547      0.402     -1.630      0.103      -1.443       0.133
Team_LAA             -0.3448      0.349     -0.987      0.324      -1.031       0.341
Team_LAD              0.5128      0.393      1.303      0.193      -0.259       1.285
Team_MIA             -0.7155      0.385     -1.858      0.063      -1.471       0.040
Team_MIL              0.0146      0.368      0.040      0.968      -0.708       0.738
Team_MIN             -0.7104      0.422     -1.683      0.093      -1.539       0.118
Team_NYM             -0.4785      0.366     -1.309      0.191      -1.196       0.239
Team_NYY             -0.2801      0.356     -0.786      0.432      -0.980       0.419
Team_OAK             -0.3243      0.411     -0.789      0.430      -1.131       0.482
Team_PHI             -0.4865      0.402     -1.211      0.226      -1.275       0.302
Team_PIT             -0.6020      0.430     -1.401      0.162      -1.445       0.241
Team_SD              -0.8274      0.402     -2.059      0.040      -1.616      -0.039
Team_SEA              0.2688      0.403      0.666      0.505      -0.523       1.060
Team_SF              -0.9562      0.384     -2.493      0.013      -1.709      -0.204
Team_STL             -0.2004      0.366     -0.547      0.585      -0.920       0.519
Team_TB              -0.6166      0.418     -1.475      0.141      -1.437

### 前5年

In [40]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016]
1633
458


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     5.646
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.94e-22
Time:                        16:54:39   Log-Likelihood:                -855.05
No. Observations:                 458   AIC:                             1800.
Df Residuals:                     413   BIC:                             1986.
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -1.8641      2.087     -0.893      0.372      -5.966       2.238
is_changeTeam        -0.9218      0.196     -4.699      0.000      -1.307      -0.536
Age                  -0.1382      0.030     -4.592      0.000      -0.197      -0.079
Age^2              6.735e-05   1.47e-05      4.583      0.000    3.85e-05    9.62e-05
AAV_dollar            0.5710      0.113      5.058      0.000       0.349       0.793
is_1year_contract     0.1625      0.249      0.652      0.515      -0.328       0.653
is_END_year          -1.2590      0.255     -4.931      0.000      -1.761      -0.757
Team_ATL             -1.3973      0.589     -2.370      0.018      -2.556      -0.239
Team_BAL             -0.3019      0.728     -0.415      0.679      -1.733       1.129
Team_BOS             -0.4918      0.568     -0.866      0.387      -1.608       0.624
Team_CHC             -0.8298      0.603     -1.376      0.170      -2.015       0.356
Team_CHW             -1.0041      0.622     -1.613      0.107      -2.228       0.219
Team_CIN             -0.7755      0.580     -1.337      0.182      -1.916       0.365
Team_CLE             -1.2859      0.585     -2.197      0.029      -2.437      -0.135
Team_COL             -1.3034      0.674     -1.935      0.054      -2.628       0.021
Team_DET             -0.6351      0.629     -1.009      0.313      -1.872       0.602
Team_HOU             -0.0263      0.716     -0.037      0.971      -1.435       1.382
Team_KC              -1.4070      0.722     -1.949      0.052      -2.826       0.012
Team_LAA             -0.1660      0.610     -0.272      0.786      -1.366       1.034
Team_LAD              0.3917      0.704      0.556      0.578      -0.992       1.775
Team_MIA             -1.0317      0.588     -1.754      0.080      -2.188       0.124
Team_MIL             -0.4416      0.602     -0.733      0.464      -1.625       0.742
Team_MIN             -1.1801      0.682     -1.729      0.084      -2.521       0.161
Team_NYM             -0.1045      0.645     -0.162      0.871      -1.373       1.164
Team_NYY             -1.0079      0.562     -1.794      0.074      -2.113       0.097
Team_OAK             -1.1920      0.681     -1.751      0.081      -2.530       0.146
Team_PHI             -0.4572      0.754     -0.606      0.545      -1.940       1.026
Team_PIT             -0.5965      0.638     -0.934      0.351      -1.851       0.658
Team_SD              -1.1876      0.656     -1.809      0.071      -2.478       0.103
Team_SEA             -0.1881      0.612     -0.307      0.759      -1.391       1.015
Team_SF              -1.1325      0.672     -1.686      0.093      -2.453       0.188
Team_STL             -0.0487      0.602     -0.081      0.936      -1.232       1.135
Team_TB              -1.0845      0.647     -1.676      0.095      -2.357

### 後5年

In [41]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2017 2018 2019 2020 2021]
934
512


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.330
Method:                 Least Squares   F-statistic:                     6.604
Date:                Sun, 05 Sep 2021   Prob (F-statistic):           1.99e-28
Time:                        16:54:51   Log-Likelihood:                -884.62
No. Observations:                 512   AIC:                             1861.
Df Residuals:                     466   BIC:                             2056.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
is_changeTeam        -0.6119      0.166     -3.679      0.000      -0.939      -0.285
Age                  -0.1818      0.027     -6.619      0.000      -0.236      -0.128
Age^2              8.846e-05   1.34e-05      6.608      0.000    6.22e-05       0.000
AAV_dollar            0.4330      0.099      4.379      0.000       0.239       0.627
is_1year_contract     0.2047      0.199      1.031      0.303      -0.186       0.595
is_END_year          -1.1734      0.208     -5.653      0.000      -1.581      -0.765
Team_ATL             -0.2267      0.428     -0.530      0.596      -1.067       0.613
Team_BAL             -1.3932      0.513     -2.716      0.007      -2.401      -0.385
Team_BOS             -0.6610      0.429     -1.540      0.124      -1.504       0.182
Team_CHC             -0.3662      0.484     -0.757      0.450      -1.317       0.585
Team_CHW             -0.2380      0.525     -0.453      0.651      -1.270       0.794
Team_CIN              0.0553      0.538      0.103      0.918      -1.002       1.112
Team_CLE             -0.2851      0.437     -0.652      0.515      -1.145       0.574
Team_COL             -1.5846      0.489     -3.242      0.001      -2.545      -0.624
Team_DET             -1.0093      0.526     -1.919      0.056      -2.043       0.024
Team_HOU              0.8759      0.445      1.969      0.050       0.002       1.750
Team_KC              -0.2228      0.471     -0.473      0.637      -1.149       0.704
Team_LAA             -0.2444      0.422     -0.579      0.563      -1.074       0.585
Team_LAD              0.7995      0.465      1.718      0.087      -0.115       1.714
Team_MIA             -0.3890      0.537     -0.724      0.470      -1.445       0.667
Team_MIL              0.5478      0.461      1.189      0.235      -0.357       1.453
Team_MIN             -0.1286      0.545     -0.236      0.813      -1.199       0.942
Team_NYM             -0.5231      0.435     -1.203      0.230      -1.378       0.331
Team_NYY              0.5958      0.473      1.259      0.209      -0.334       1.525
Team_OAK              0.3681      0.521      0.706      0.481      -0.657       1.393
Team_PHI             -0.2903      0.469     -0.619      0.536      -1.213       0.632
Team_PIT             -0.7611      0.631     -1.207      0.228      -2.000       0.478
Team_SD              -0.5034      0.516     -0.976      0.330      -1.517       0.510
Team_SEA              0.9391      0.577      1.628      0.104      -0.194       2.073
Team_SF              -0.6566      0.462     -1.422      0.156      -1.564       0.251
Team_STL             -0.0810      0.461     -0.176      0.861      -0.988       0.826
Team_TB              -0.0117      0.560     -0.021      0.983      -1.112       1.089
Team_TEX              0.0911      0.508      0.179      0.858      -0.907